In [3]:
%load_ext autoreload

In [4]:
%autoreload 2

In [5]:
%matplotlib inline
import sys
sys.path.append('../')
import RNA 
from utils.sequence_utils import generate_random_mutant
from utils.model_architectures import Linear, NLNN, CNNa
from models.Noisy_models.Neural_network_models import NN_model
from models.Ground_truth_oracles.RNA_landscape_models import RNA_landscape_constructor
from models.Noisy_models.Ensemble import Ensemble_models
from evaluators.Evaluator import Evaluator
from models.Ground_truth_oracles.TF_binding_landscape_models import *

LANDSCAPE_TYPES ={"RNA": [3,2], 'TF':[1]}
LANDSCAPE_TYPES2 = {'RNA': [], 'TF' : ['SIX6_REF_R1']}
mutation_rate = 0.5
RNA_alphabet = "UCGA" 
DNA_alphabet = "TCGA"
landscape_constructor = RNA_landscape_constructor()
#load a pre-specified landscapes  (here, index 0), you can also specify to just take a few of them
landscape_constructor.load_landscapes("../data/RNA_landscapes/RNA_landscape_config.yaml", landscapes_to_test=[0])
landscape_generator = landscape_constructor.generate_from_loaded_landscapes()
landscape_object=next(landscape_generator)
wt = landscape_object['starting_seqs']['startRNAL14_3']
landscape_oracle = landscape_object['landscape_oracle']

training_sequences = [wt]
cnn_architecture=CNNa(len(wt), alphabet=RNA_alphabet)
noisy_nn_model=NN_model(landscape_oracle, cnn_architecture)
noisy_nn_model.reset()
noisy_nn_model.update_model(training_sequences)

Using TensorFlow backend.


B1L14RNA1 loaded





../models/Ground_truth_oracles/RNA_landscape_models.py:21: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  self.loaded_landscapes = yaml.load(cfgfile)




Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.









/home/alexander/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_regression.py:582: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)


## DQN Explorer

In [19]:
from explorers.dqn_explorer import DQN_Explorer 

num_of_rounds = 5

# reset model if it already exists 
training_sequences = [wt]
cnn_architecture=CNNa(len(wt), alphabet=RNA_alphabet)
noisy_nn_model=NN_model(landscape_oracle, cnn_architecture)
noisy_nn_model.reset()
noisy_nn_model.update_model(training_sequences)

# evaluate DQN Explorer 
dqn_explorer = DQN_Explorer(train_epochs=10)
dqn_explorer.debug = True
dqn_explorer.set_model(noisy_nn_model)
dqn_explorer.run(num_of_rounds, verbose=True)

/home/alexander/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_regression.py:582: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)


round: 0, cost: 1, evals: 0, top: 0.14554841726711917
round: 1, cost: 101, evals: 854, top: 0.590432237476802
round: 2, cost: 201, evals: 1656, top: 0.6590871488113138
round: 3, cost: 301, evals: 2355, top: 0.6975338886827324
round: 4, cost: 401, evals: 2840, top: 0.6975338886827324


In [21]:
landscape_constructor=TF_binding_landscape_constructor()
landscape_constructor.load_landscapes(landscapes_to_test = ['SIX6_REF_R1'])
landscape_generator = landscape_constructor.generate_from_loaded_landscapes()
landscape_object = next(landscape_generator)
wt = landscape_object['starting_seqs']['TF0']
landscape_oracle = landscape_object['landscape_oracle']

# reset model if it already exists 
training_sequences = [wt]
cnn_architecture=CNNa(len(wt), alphabet=DNA_alphabet)
noisy_nn_model=NN_model(landscape_oracle, cnn_architecture)
noisy_nn_model.reset()
noisy_nn_model.update_model(training_sequences)

# evaluate DQN Explorer 
dqn_explorer = DQN_Explorer(alphabet=DNA_alphabet, train_epochs=10, virtual_screen=20)
dqn_explorer.debug = True
dqn_explorer.set_model(noisy_nn_model)
dqn_explorer.run(num_of_rounds, verbose=True)

1 TF landscapes loaded.


/home/alexander/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_regression.py:582: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)


round: 0, cost: 1, evals: 0, top: 0.6781429101554447
round: 1, cost: 101, evals: 1498, top: 0.9804354100523647
round: 2, cost: 197, evals: 2773, top: 0.9929802498659959
round: 3, cost: 286, evals: 3773, top: 0.9929802498659959
round: 4, cost: 357, evals: 4548, top: 0.9929802498659959


In [4]:
from explorers.dqn_explorer import DQN_Explorer 

dqn_explorer_prod = DQN_Explorer(train_epochs=10)
dqn_explorer_prod.debug = False

In [23]:
evaluator_dqn=Evaluator(dqn_explorer_prod,landscape_types=LANDSCAPE_TYPES, path="../simulations/eval/")
evaluator_dqn.evaluate_for_landscapes(evaluator_dqn.adaptivity, num_starts=1)

loading landscapes RNA: [3, 2], TF:[1]
B1L14RNA2 loaded
B1L100RNA1 loaded
1 TF landscapes loaded.
loading complete
Running on B1L14RNA2
start seq startRNAL14_0


../models/Ground_truth_oracles/RNA_landscape_models.py:21: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  self.loaded_landscapes = yaml.load(cfgfile)


Running on B1L100RNA1
start seq startRNAL100_0
Running on CRX_R41W_R1
start seq TF0


In [5]:
evaluator_dqn=Evaluator(dqn_explorer_prod,landscape_types=LANDSCAPE_TYPES, path="../simulations/eval/")
evaluator_dqn.evaluate_for_landscapes(evaluator_dqn.efficiency, num_starts=1)

loading landscapes RNA: [3, 2], TF:[1]
B1L14RNA2 loaded
B1L100RNA1 loaded


../models/Ground_truth_oracles/RNA_landscape_models.py:21: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  self.loaded_landscapes = yaml.load(cfgfile)


1 TF landscapes loaded.
loading complete
Running on B1L14RNA2
start seq startRNAL14_0
Running on B1L100RNA1
start seq startRNAL100_0
Running on CRX_R41W_R1
start seq TF0


In [6]:
evaluator_dqn=Evaluator(dqn_explorer_prod,landscape_types=LANDSCAPE_TYPES, path="../simulations/eval/")
evaluator_dqn.evaluate_for_landscapes(evaluator_dqn.scalability, num_starts=1)

loading landscapes RNA: [3, 2], TF:[1]
B1L14RNA2 loaded
B1L100RNA1 loaded
1 TF landscapes loaded.
loading complete
Running on B1L14RNA2
start seq startRNAL14_0
B1L14RNA2,100,1,0.2407999038696289

B1L14RNA2,100,10,2.756357192993164

B1L14RNA2,100,100,11.276934385299683

B1L14RNA2,1000,1,2.217010021209717

B1L14RNA2,1000,10,12.318877935409546

B1L14RNA2,1000,100,112.82714009284973

Running on B1L100RNA1
start seq startRNAL100_0
B1L100RNA1,100,1,3.2316184043884277

B1L100RNA1,100,10,14.342337608337402

B1L100RNA1,100,100,130.99797320365906

B1L100RNA1,1000,1,25.335120677947998

B1L100RNA1,1000,10,435.85530281066895

B1L100RNA1,1000,100,1268.5383896827698

Running on CRX_R41W_R1
start seq TF0
CRX_R41W_R1,100,1,0.20346808433532715

CRX_R41W_R1,100,10,1.070932149887085

CRX_R41W_R1,100,100,10.134784698486328

CRX_R41W_R1,1000,1,1.966050148010254

CRX_R41W_R1,1000,10,10.439759969711304

CRX_R41W_R1,1000,100,96.29717946052551



In [8]:
evaluator_dqn=Evaluator(dqn_explorer_prod,landscape_types=LANDSCAPE_TYPES2, path="../simulations/eval/")
evaluator_dqn.evaluate_for_landscapes(evaluator_dqn.adaptivity, num_starts=1)
evaluator_dqn.evaluate_for_landscapes(evaluator_dqn.efficiency, num_starts=1)
evaluator_dqn.evaluate_for_landscapes(evaluator_dqn.scalability, num_starts=1)

loading landscapes RNA: [], TF:['SIX6_REF_R1']
1 TF landscapes loaded.
loading complete
Running on SIX6_REF_R1
start seq TF0


## BO Explorer

In [8]:
%autoreload 2
from explorers.bo_explorer import BO_Explorer

num_of_rounds = 5
arch1 = NLNN(len(wt), hidden_dims=10, epochs=20)
arch2 = NLNN(len(wt), hidden_dims=20, epochs=20)
arch3 = NLNN(len(wt), hidden_dims=30, epochs=20)
arch4 = NLNN(len(wt), hidden_dims=40, epochs=20)
# ensemble we will use for BO Explorer
nn_model_1 = NN_model(landscape_oracle, arch1)
nn_model_2 = NN_model(landscape_oracle, arch2)
nn_model_3 = NN_model(landscape_oracle, arch3)
nn_model_4 = NN_model(landscape_oracle, arch4)
models = [nn_model_1, nn_model_2, nn_model_3, nn_model_4]
ensemble_model = Ensemble_models(models)
ensemble_model.update_model(training_sequences)

/home/alexander/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_regression.py:582: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/home/alexander/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_regression.py:582: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/home/alexander/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_regression.py:582: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/home/alexander/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_regression.py:582: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)


In [9]:
# evaluate BO Explorer 
bo_explorer = BO_Explorer()
bo_explorer.debug = True
bo_explorer.set_model(ensemble_model)
bo_explorer.run(num_of_rounds, verbose=True)

round: 0, cost: 1, evals: 0.0, top: 0.14554841726711917
round: 1, cost: 101, evals: 3975.0, top: 0.55747785908442
round: 2, cost: 201, evals: 7819.0, top: 0.55747785908442
round: 3, cost: 301, evals: 13007.0, top: 0.6069094266729929
round: 4, cost: 401, evals: 22512.0, top: 0.6151479950813181


In [10]:
bo_explorer_prod = BO_Explorer()
bo_explorer_prod.debug = False

evaluator_bo=Evaluator(bo_explorer,landscape_types=LANDSCAPE_TYPES, path="../simulations/eval/")
evaluator_bo.evaluate_for_landscapes(evaluator_bo.adaptivity, num_starts=1)

loading landscapes RNA: [3, 2], TF:[1]
B1L14RNA2 loaded
B1L100RNA1 loaded
1 TF landscapes loaded.
loading complete
Running on B1L14RNA2
start seq startRNAL14_0


../models/Ground_truth_oracles/RNA_landscape_models.py:21: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  self.loaded_landscapes = yaml.load(cfgfile)


Running on B1L100RNA1
start seq startRNAL100_0
Running on CRX_R41W_R1
start seq TF0
